In [1]:
import nltk
from nltk.corpus import brown
from nltk.util import ngrams
from collections import Counter
import pandas as pd
import numpy as np

D:\ProgramData\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
D:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
D:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

from string import punctuation

punctuation = set(list(punctuation))

In [3]:
words = brown.words()
words = [word.lower() for word in words]
words = [word for word in words if word.isalpha()]
words = [word for word in words if word not in punctuation]

In [4]:
f_words = nltk.FreqDist(words)

In [5]:
most_common = sorted(f_words.most_common(5000), key= lambda tup: tup[1])
most_common_words = [tup[0] for tup in most_common]

In [6]:
table_words = ['cord', 'smile', 'hill', 'woodland', 'rooster', 'voyage', 'car', 'journey', 'noon',
         'string', 'cemetery', 'mound', 'fruit', 'furnace', 'glass', 'jewel', 'autograph', 'shore',
         'magician', 'oracle', 'automobile', 'wizard', 'crane', 'implement', 'mound', 'stove',
         'brother', 'lad', 'grin', 'implement', 'sage', 'wizard', 'asylum', 'fruit', 'oracle',
         'bird', 'sage', 'asylum', 'monk', 'graveyard', 'crane', 'madhouse', 'bird', 'cock', 'glass',
         'magician', 'rooster', 'brother', 'monk', 'food', 'fruit', 'cushion', 'jewel', 'asylum',
         'madhouse', 'monk', 'slave', 'cemetery', 'magician', 'wizard', 'furnace', 'stove', 'asylum',
         'coast', 'forest', 'hill', 'mound', 'grin', 'lad', 'cord', 'string', 'shore', 'woodland',
         'glass', 'tumbler', 'monk', 'oracle', 'grin', 'smile', 'boy', 'sage', 'serf', 'slave',
         'automobile', 'cushion', 'journey', 'voyage', 'mound', 'shore', 'autograph', 'signature',
         'lad', 'wizard', 'coast', 'shore', 'forest', 'graveyard', 'forest', 'woodland', 'food',
         'rooster', 'implement', 'tool', 'cemetery', 'woodland', 'cock', 'rooster', 'shore', 'voyage',
         'boy', 'Ind', 'bird', 'woodland', 'cushion', 'coast', 'hill', 'cemetery', 'furnace',
         'implement', 'automobile', 'crane', 'rooster', 'midday', 'nооn', 'gem', 'jewel', 'boy',
         'pillow', 'graveyard', 'car']

In [7]:
final_words = set(table_words + most_common_words)

In [8]:
bi_grams = ngrams(words, 2)
bigram_counter = Counter(bi_grams)
final_words_list = list(final_words)
count_df = pd.DataFrame(columns=final_words_list, index=final_words_list)
pmi_df  = pd.DataFrame(columns=final_words_list, index=final_words_list)

In [9]:
bigram_set = set((w1,w2) for w1,w2 in ngrams(words, 2))
bigrams_list = [(w1,w2) for w1,w2 in ngrams(words, 2)]

In [10]:
for word1,word2 in bigram_set:
    if word1 in final_words and word2 in final_words:
        bigram_count = bigram_counter[(word1, word2)]
        count_df[word1].loc[word2] = bigram_count
        bigram_prob = bigram_count/len(bigrams_list)
        word1_prob = f_words[word1]/len(words)
        word2_prob = f_words[word2]/len(words)
        pmi_df[word1].loc[word2] = np.log2(bigram_prob/(word1_prob*word2_prob))

In [11]:
ppmi_df = pmi_df.copy()
ppmi_df[ppmi_df < 0] = 0

In [12]:
word_to_index = {w:i for i,w in enumerate(final_words_list)}

In [13]:
from sklearn.decomposition import PCA

result = PCA(300).fit_transform(ppmi_df.fillna(0).values)

In [14]:
lsa_mat_10  = result[:,:10]
lsa_mat_100 = result[:,:100]
lsa_mat_300 = result[:,:]

In [15]:
import pickle
with open("lsa_word_vec_with_word_index300_final1.pkl", 'wb') as f:
    pickle.dump((lsa_mat_300, word_to_index), f)

In [16]:
n11_grams = ngrams(words, 5)
n11gram_counter = Counter(n11_grams)
pmi_5_df  = pd.DataFrame(columns=final_words_list, index=final_words_list)

In [17]:
n11gram_set = set(tuple(gram) for gram in ngrams(words, 5))
n11grams_list = [tuple(gram) for gram in ngrams(words, 5)]

In [18]:
p5_counter = {}
for word_gram in n11grams_list:
    word1 = word_gram[2]
    for i, word2 in enumerate(word_gram):
        if i != 2:
            p5_counter[(word1, word2)] = p5_counter.get((word1, word2), 0) + 1/np.abs(i-2)

In [19]:
for word1,word2 in p5_counter:
    if word1 in final_words and word2 in final_words:
        cooccur_count = p5_counter[(word1, word2)]
        cooccur_prob = cooccur_count/len(n11grams_list)
        word1_prob = f_words[word1]/len(words)
        word2_prob = f_words[word2]/len(words)
        pmi_5_df[word1].loc[word2] = np.log2(cooccur_prob/(word1_prob*word2_prob))

In [20]:
result = PCA(300).fit_transform(pmi_5_df.fillna(0).values)

In [21]:
import pickle
with open("lsa_word_vec_with_word_index300_final2_con5.pkl", 'wb') as f:
    pickle.dump((result, word_to_index), f)